In [28]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

In [29]:
symbols = pd.read_csv('../dataset/symbols_valid_meta.csv')
stock_paths = {}
stock_dfs = {}
stock_rows = {}
stock_symbol_to_name_map = {}

In [30]:
symbols_list = list(symbols['Symbol'])

In [31]:
for _, row in symbols[['Symbol', 'Security Name']].iterrows():
    symbol, name = row
    if symbol not in stock_symbol_to_name_map:
        stock_symbol_to_name_map[symbol] = name

In [32]:
for symbol in symbols_list:
    if symbol not in stock_paths:
        stock_paths[symbol] = f'../dataset/stocks/{symbol}.csv'


In [33]:
for symbol, stock_path in stock_paths.items():
    if symbol not in stock_dfs:
        try:
            stock_dfs[symbol] = pd.read_csv(stock_path)
        except:
            continue


In [34]:
for stock_name, stock_df in stock_dfs.items():
    if stock_name not in stock_rows:
        stock_rows[stock_name] = len(stock_df)

In [35]:
for stock_name, stock_df in stock_dfs.items():
    stock_df['Date'] = pd.to_datetime(stock_df['Date'])

In [36]:
def get_rows_within_date_range(df, start_date, end_date):
    return df.loc[((df['Date'] >= start_date) & (df['Date'] <= end_date))]

In [37]:
def get_start_and_end_date_of_stock(df):
    dates = list(df['Date'])
    return (dates[0], dates[-1])

In [38]:
def populate_open_values_efficient(stocks):
    open_value_stocks = pd.DataFrame(columns=['Date'])
    stocks_to_analyze = {}
    stocks_to_analyze_rows = {}
    stock_names = []
    
    for stock in stocks:
        if stock not in stocks_to_analyze:
            stocks_to_analyze[stock] = stock_dfs[stock]
    
    #get stock rows for our stocks
    for stock in stocks:
        if stock not in stocks_to_analyze_rows:
            stocks_to_analyze_rows[stock] = stock_rows[stock]
    
    # sort stock_rows
    stock_with_smallest_date_range = sorted(stocks_to_analyze_rows.items(), key=lambda x: x[1])[0]
    
    benchmark_stock_name, benchmark_stock_rows = stock_with_smallest_date_range
    
    start_date, end_date = get_start_and_end_date_of_stock(stock_dfs[benchmark_stock_name])
    
    open_value_stocks['Date'] = stock_dfs[benchmark_stock_name]['Date']
    to_merge = [open_value_stocks.reset_index(drop=True)]
    
    for stock_name, stock_df in stocks_to_analyze.items():
        new_col = pd.DataFrame(get_rows_within_date_range(stock_df, start_date, end_date)['Open']).rename(columns={'Open': stock_name})
        if len(new_col) == benchmark_stock_rows: # make sure the current stock has the same rows as the benchmark stock
            to_merge.append(new_col.reset_index(drop=True))
            
            stock_names.append(stock_name)
    
    open_value_stocks = pd.concat(to_merge, axis=1)
    
    return (open_value_stocks, stock_names)


In [39]:
def populate_close_values_efficient(stocks):
    close_value_stocks = pd.DataFrame(columns=['Date'])
    stocks_to_analyze = {}
    stocks_to_analyze_rows = {}
    stock_names = []
    
    # populate stocks to analyze
    for stock in stocks:
        if stock not in stocks_to_analyze:
            stocks_to_analyze[stock] = stock_dfs[stock]
    
    # populate the row counts of the stocks to analyze
    for stock in stocks:
        if stock not in stocks_to_analyze_rows:
            stocks_to_analyze_rows[stock] = stock_rows[stock]
    
    # sort stock_rows
    stock_with_smallest_date_range = sorted(stocks_to_analyze_rows.items(), key=lambda x: x[1])[0]
    
    benchmark_stock_name, benchmark_stock_rows = stock_with_smallest_date_range
    
    start_date, end_date = get_start_and_end_date_of_stock(stock_dfs[benchmark_stock_name])
    
    close_value_stocks['Date'] = stock_dfs[benchmark_stock_name]['Date']
    to_merge = [close_value_stocks.reset_index(drop=True)]
    
    for stock_name, stock_df in stocks_to_analyze.items():
        new_col = pd.DataFrame(get_rows_within_date_range(stock_df, start_date, end_date)['Close']).rename(columns={'Close': stock_name})
        if len(new_col) == benchmark_stock_rows: # make sure the current stock has the same rows as the benchmark stock
            to_merge.append(new_col.reset_index(drop=True))
            stock_names.append(stock_name)
    
    close_value_stocks = pd.concat(to_merge, axis=1)
    
    return close_value_stocks


In [40]:
def KMeansArrayedStocks(stocks, k=5, max_iter=1000):
    # get open values for each stock
    all_stocks_open, stock_symbols = populate_open_values_efficient(stocks)
    all_stocks_close = populate_close_values_efficient(stocks)
    
    display(stock_symbols)
    
    stock_names = [stock_symbol_to_name_map[stock] for stock in stock_symbols]
    
    all_stocks_open.drop('Date', axis=1, inplace=True)
    all_stocks_close.drop('Date', axis=1, inplace=True)
    
    open_values = np.array(all_stocks_open.T)
    close_values = np.array(all_stocks_close.T)
    
    # calculate daily movements
    daily_movements = close_values - open_values
    
    
    # create the pipeline
    normalizer = Normalizer()
    clustering_model = KMeans(n_clusters=k, max_iter=max_iter)
    pipeline = make_pipeline(normalizer, clustering_model)
    pipeline.fit(daily_movements)
    clusters = pipeline.predict(daily_movements)
    
    # get results
    results = pd.DataFrame({
        'clusters': clusters,
        'symbols': stock_names
    }).sort_values(by=['clusters'], axis=0)
    
    display(results)
    

In [50]:
FAANG = ['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOGL']
automobiles = ['GM', 'F', 'HMC', 'TM']
financial = ['JPM', 'MS', 'GS']
telecom = ['TMUS', 'RCI', 'USM']
entertainment = ['EA', 'ATVI', 'DIS', 'IMAX']
defense = ['LMT', 'CAE', 'GD', 'LHX']
stocks_to_analyze = FAANG + automobiles + financial + telecom + entertainment + defense

KMeansArrayedStocks(stocks_to_analyze, k=8, max_iter=2000)

['FB',
 'AAPL',
 'AMZN',
 'NFLX',
 'GOOGL',
 'GM',
 'F',
 'HMC',
 'TM',
 'JPM',
 'GS',
 'TMUS',
 'RCI',
 'USM',
 'EA',
 'ATVI',
 'DIS',
 'IMAX',
 'LMT',
 'CAE',
 'GD',
 'LHX']

,clusters,symbols
8,0,Toyota Motor Corporation Common Stock
7,0,"Honda Motor Company, Ltd. Common Stock"
5,0,General Motors Company Common Stock
6,0,Ford Motor Company Common Stock
21,1,"L3Harris Technologies, Inc. Common Stock"
18,1,Lockheed Martin Corporation Common Stock
20,1,General Dynamics Corporation Common Stock
0,2,"Facebook, Inc. - Class A Common Stock"
3,2,"Netflix, Inc. - Common Stock"
2,2,"Amazon.com, Inc. - Common Stock"
